In [ ]:
import requests
import json

In [ ]:
#text=['Oh man oh man','where is this world going','Ia m soo hungry','I dont like you Cira', 'Junction food sucks','I dont know what I wanna do','How are you','What shall I do today']

In [ ]:
import indicoio

In [ ]:
#jenny's api key for indicoio
indicoio.config.api_key = 'd3b7a4d2761c18a403b04e1fe7802e43'

In [ ]:
# get json file of community cira bot
response=requests.get('https://api.vk.com/method/groups.getLongPollServer?group_id=174367830&access_token=fbdc5bd3cff381a236f0f30a1b58982ca4ddf1508e03a6513af837d67184c6ffa064d2a6737a897bc012e&v=5.92')
community_resp=response.json()
print(community_resp)
print(community_resp['response']['key'])

In [ ]:
messages=requests.get('https://lp.vk.com/wh174367830?act=a_check&key=%s&ts=1&wait=%s'%(community_resp['response']['key'],community_resp['response']['ts']))
messages_json=messages.json()

In [ ]:
#messages from the community bot json file
messages_json

In [ ]:
#text_lastmess=messages_json['updates'][-1]['object']['text']

In [ ]:
#text_lastmess

In [ ]:
#emotion_lastmess=indicoio.emotion(text_lastmess)

In [ ]:
# emot_dic={}
# emot_dic['anger']=['calm down']

In [ ]:
# emotion_lastmess

In [ ]:
# resp_mes = "?"
# if emotion_lastmess['anger']>0.3:
#     resp_mes=emot_dic['anger']
# print(resp_mes)

## Mood Classifier (from user's wall posts)

In [ ]:
requests.post('https://api.vk.com/method/messages.send?peer_id=518240818&random_id=1&message=%s&access_token=fbdc5bd3cff381a236f0f30a1b58982ca4ddf1508e03a6513af837d67184c6ffa064d2a6737a897bc012e&v=5.92'%resp_mes)

#### Grab the user from the community bot

In [ ]:
#get the last user from community bot
lastuser = messages_json['updates'][-1]['object']['from_id']

#getposts = indicoio.emotion(text_lastmess)

#### Get user profile and wall posts

In [ ]:
#get json file of last user's profile
profile = requests.get('https://api.vk.com/method/wall.get?owner_id='+str(lastuser)+'&access_token=30691f8630691f8630691f8607300e355a3306930691f866b917e8e8459c4931e49642f&v=5.92')
profile = profile.json()

In [ ]:
#get the wall posts from the last user's profile
wallposts = profile['response']['items']
wallposts

### Analyze all wall posts for mood

In [ ]:
#DICTIONARIES
# moods - dictionary of users : dictionary of 5 moods and their average score
# counter - for incremental averages of moods
score = 0
moods = {}
#     {'owner_id': 
#          {'joy': score, 'surprise': score, 'anger': score, 'fear': score, 'sadness': score}
#         }
counter = {'owner_id': 0}

In [ ]:
def updateMood(user, feel, newscore):
        currscore = moods[user][feel]
        val = ((newscore - currscore) / counter[user])
        moods[person][feel] += val
        
#scores emotions for each comment
for each in wallposts:
    user = each['owner_id']
    if user not in counter:
        counter[user] = 0
        moods[user] = {'joy': 0, 'surprise': 0, 'anger': 0, 'fear': 0, 'sadness': 0}
    
    counter[user] += 1
    emotions = indicoio.emotion(each['text'])
#     print("EMOTION INFO:  ", emotions)
    
    #incremental average moods
    for feel in emotions:
        updateMood(user, feel, emotions[feel])
#     print("EMOTION AVERAGE:  ", moods)
#     print()

print(moods)
print(counter)

### Check for the mood

In [ ]:
#active bot output

import operator

def getConf(mood):
    if mood > 0.5:
        return 2
    elif mood > 0.2:
        return 1
    else:
        return 0

def checkMood(user):
    maxMood = max(moods[user].items(), key=operator.itemgetter(1))[0]
    moodConf = getConf(moods[user][maxMood])
    if moodConf is 0:
        return (neutral, -1)
    return(maxMood, moodConf)

checkMood(518274967)